In [ ]:
#This is a hack to make the %%cpp work inside mybinder.org - ignore
from ROOT import TObject

In [ ]:
%%cpp
//Create a c++ object representing a LorentzVector
//More details at https://root.cern.ch/doc/master/classTLorentzVector.html
TLorentzVector v;
//Print the vectors components - should see all components initialised to zero
v.Print()

In [ ]:
%%cpp
//We see this does not work, because 1 and 2 are integers.
//The template arguments to DeltaPhi must be classes that implement specific methods.
double test = ROOT::Math::VectorUtil::DeltaPhi(1,2)

In [ ]:
%%cpp
//We try deltaphi again, this time with instances of classes that do implement the required methods.
TLorentzVector mark1(0.0,0.0,0.0,0.0);
TLorentzVector mark2(0.0,0.0,0.0,0.0);
double test = ROOT::Math::VectorUtil::DeltaPhi(mark1,mark2);
test

In [ ]:
#Now do the same in python
#Don't forget in python we must import each ROOT C++ class we want to use.
from ROOT import TLorentzVector
mark1 = TLorentzVector(0.0,0.0,0.0,0.0)
mark2 = TLorentzVector(0.0,0.0,0.0,0.0)
from ROOT import Math
#Won't work
Math.VectorUtil.DeltaPhi(1,2)

In [ ]:
#Will work
Math.VectorUtil.DeltaPhi(mark1,mark2)

In [ ]:
#Task 1
#Download https://nbviewer.jupyter.org/url/root.cern/doc/master/notebooks/mathcoreVectorCollection.C.nbconvert.ipynb
#and run it.
#Note I spotted the last two cells are missing the %%cpp!

In [ ]:
%%cpp
//Magic numbers
//No need to write "double M = 0.13957;" in above example
TDatabasePDG db;
//If everyone uses it, all code uses consistent numbers in calculations - easier to debug your code
//with respect to colleagues codes etc
db.GetParticle(211)->Mass()

In [ ]:
from ROOT import TFile
#You can find more ATLAS open datasets at http://opendata.atlas.cern/extendedanalysis/datasets.php
#You can open this directly via the URL - note its likely faster to have a local copy wherever you run
#the notebook
zTauTauFile = TFile.Open("http://opendata.atlas.cern/release/samples/MC/mc_147772.Ztautau.root")
myTree = zTauTauFile.Get("mini;1")
from ROOT import TCanvas
can = TCanvas()
can.cd()
myTree.Draw("lep_pt")
#If reading over URL, be patient - will take a few minutes for plot to appear!
can.Draw()

In [ ]:
#Task 2
#Use the Ztautau open data to plot the invariant mass of ZCandidates
#Hints: The Z always decays to two tau particles in this Monte Carlo Simulation. 
#Hints: The reconstructed tau candidates are represented by the lep_ variables.